In [1]:
import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

import numpy as np
import matplotlib as plt

import IPythonMagic
from Timer import Timer

In [2]:
%setup_logging

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


In [3]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 10392 / 11441 MB available
Created context handle <53765520>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/andrea_andreolli/MilanoGPU2018/notebooks/cuda_cache


In [8]:
kernel_src = """

__global__ void shMemReduction(float* output, float* input, int size) {
    
    // First we stride thorugh global memory and compute the maximum for every thread
    int gid = blockIdx.x * blockDim.x + threadIdx.x; //blockId.x is always zero because I use one block only
    
    // I didn't really get the trick here, the point is that this code accesses global GPU memory efficiently
    float max_value = -99999999.99; // FIX ME WITH CORRECT VALUE
    for(int i = threadIdx.x; i < size; i = i + blockDim.x) {
        max_value = fmaxf(max_value, input[i]);
    }
    
    // Temporary write to memory to check if things work so far
    output[threadIdx.x] = max_value;
    
    // Store the per-thread maximum in shared memory
    __shared__ float max_shared[32];
    max_shared[threadIdx.x] = max_value;
    
    // Synchronise so that all thread see the same memory
    __syncthreads();
    
    // Find the maximum in memory: 32 to 16
    if (threadIdx.x < 16) {
         max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
    
    // Find the maximum in memory: 16 to 8
    if (threadIdx.x < 8) {
         max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
   
   // Find the maximum in memory: 8 to 4
    if (threadIdx.x < 4) {
         max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
    
    // Find the maximum in memory: 4 to 2
    if (threadIdx.x < 2) {
         max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
    
    // Find the maximum in memory
    if (threadIdx.x = 0) {
         max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
         output[0] = max_shared[0];
    }
    
    // Finally write output
}

"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shMemReduction")

CompileError: nvcc compilation of /tmp/tmpvex7b4d0/kernel.cu failed
[command: nvcc --cubin -arch sm_37 -I/home/ubuntu/.local/lib/python3.6/site-packages/pycuda/cuda kernel.cu]
[stderr:
kernel.cu(46): error: expression must be a modifiable lvalue

kernel.cu(7): warning: variable "gid" was declared but never referenced

1 error detected in the compilation of "/tmp/tmpxft_00001204_00000000-6_kernel.cpp1.ii".
]

In [5]:
n = 64
a = np.random.random((1,n)).astype(np.float32)
print(a)
num_threads = 32
b = np.empty((1,num_threads))

a_g = GPUArray(a.shape,a.dtype)
a_g.set(a)
b_g = GPUArray(b.shape,b.dtype)

[[0.05676738 0.88570774 0.8356549  0.04014112 0.20502482 0.73145306
  0.46846572 0.07788982 0.0980249  0.19718513 0.12497357 0.54839116
  0.44450334 0.54450774 0.06586034 0.01848972 0.09302702 0.9707608
  0.6441077  0.18992673 0.70981276 0.6398528  0.8103699  0.51682216
  0.4633676  0.39644143 0.5802235  0.769677   0.71685123 0.29946277
  0.22802688 0.9052712  0.9620451  0.6985464  0.61725545 0.23405337
  0.9067984  0.7051695  0.46646544 0.01990609 0.6253213  0.35327104
  0.33566865 0.96801126 0.4930374  0.5540883  0.60538465 0.04287912
  0.47265834 0.58728296 0.21084785 0.6005624  0.9926177  0.35323107
  0.2935759  0.51132464 0.2672205  0.17414628 0.362874   0.34549886
  0.67795324 0.18047003 0.72376376 0.01169576]]


In [6]:
block_size = (num_threads, 1, 1)
grid_size = (1,1,1)

kernel_function(b_g,a_g, np.int32(n), grid=grid_size, block=block_size)

b_g.get(b)

print(a)
print(b)

[[0.05676738 0.88570774 0.8356549  0.04014112 0.20502482 0.73145306
  0.46846572 0.07788982 0.0980249  0.19718513 0.12497357 0.54839116
  0.44450334 0.54450774 0.06586034 0.01848972 0.09302702 0.9707608
  0.6441077  0.18992673 0.70981276 0.6398528  0.8103699  0.51682216
  0.4633676  0.39644143 0.5802235  0.769677   0.71685123 0.29946277
  0.22802688 0.9052712  0.9620451  0.6985464  0.61725545 0.23405337
  0.9067984  0.7051695  0.46646544 0.01990609 0.6253213  0.35327104
  0.33566865 0.96801126 0.4930374  0.5540883  0.60538465 0.04287912
  0.47265834 0.58728296 0.21084785 0.6005624  0.9926177  0.35323107
  0.2935759  0.51132464 0.2672205  0.17414628 0.362874   0.34549886
  0.67795324 0.18047003 0.72376376 0.01169576]]
[[2.28774221e-03 5.89911976e-08 4.15832328e-04 7.16644537e-12
  1.24423354e-06 5.81320466e-03 5.69278853e-05 5.61912508e-14
  5.98505231e-03 9.82054769e-05 1.51079759e-04 3.87315303e-05
  3.21603010e-06 6.42007797e-04 3.77371008e-07 2.89909960e-03
  0.00000000e+00 0.000000